# 1 サンプルデータの取得

Fashion-MNIST（ファッションエムニスト）と呼ばれる、ファッション関連の画像データのサンプルを取得

In [1]:
import tensorflow as tf

data = tf.keras.datasets.fashion_mnist.load_data()

In [2]:
# data

トレーニングデータとテストデータを取得

In [3]:
(x_train, y_train), (x_test, y_test) = data

In [4]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

# 2 データ前処理


## 2.1 One-hotベクトル

1 -> [0,1,0,0,・・・・,0] \
2 -> [0,0,1,0,・・・・,0] \

データ数は60000。ラベルは0~9の10種類でそれぞれに衣料品の種類が割り当てられている。


In [5]:
print(y_train, len(y_train), set(y_train))

[9 0 0 ... 3 0 5] 60000 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


データの前処理を行うための便利な関数が含まれている`utils`を使う




In [6]:
from tensorflow.keras import utils

In [7]:
y_train = utils.to_categorical(y_train)
y_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
y_test = utils.to_categorical(y_test)

## 2.2 スケーリング

画像データの値を255で割る

In [9]:
x_train = x_train / 255
x_test = x_test / 255

# 3 ニューラルネットワーク構造の定義

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense



## 3.1 層の定義
層を追加する時はaddメゾットで追加する

In [11]:
model = Sequential()

In [12]:
# 入力層　28x28ピクセルのデータを入力
# Flatten()入力データを1次元データに変換する層
model.add(Flatten(input_shape=(28, 28)))

# 隠れ層 ノード数128 活性化関数:ReLU関数(正の値はそのまま、負の値は0に変換)
model.add(Dense(128, activation='relu'))

# 出力層　ノード数10(=ラベル数) 活性化関数:ソフトマックス関数(全てのノード値を足して1にする)
model.add(Dense(10, activation='softmax'))

 ## 3.2 モデルの確認

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 4 損失関数、最適化方法、評価指標の設定

損失関数：カテゴリ交差エントロピー誤差 \
最適化方法：Adam（アダム） \
評価指標：正解率

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

# 5 ニューラルネットワークの学習


学習データ:60000データ　\
バッチサイズ:32 データセットの分割数\
エポック:5 1エポックあたりのバッチ数は60000/32 = 1875データ

In [15]:
model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4981 - accuracy: 0.8253
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3794 - accuracy: 0.8613
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3376 - accuracy: 0.8770
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3141 - accuracy: 0.8846
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2977 - accuracy: 0.8896


# 6 ニューラルネットワークの評価

テストデータを使って、正解率を計算 \
[損失関数, 正解率]

In [16]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.8669


[0.364831805229187, 0.8669000267982483]

# 7 未知データの予測

In [17]:
from tensorflow.keras.preprocessing.image import load_img

テスト画像データの読み込み

In [18]:
img = load_img('/content/unknown_sneaker.png', target_size=(28, 28), color_mode='grayscale')
img

In [19]:
from tensorflow.keras.preprocessing.image import img_to_array

In [20]:
# 画像データをimg_to_array関数で画像をarray形式に変換
array = img_to_array(img)
# array

In [21]:
array.shape

(28, 28, 1)

In [22]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [23]:
# 画像データの形を変換
import numpy as np

array = array.reshape((1, 28, 28))
array.shape

(1, 28, 28)

In [24]:
# スケーリング
array = array / 225

出力された10の数値のうち最も大きいのは8つ目の要素。\
入力した画像データは、ラベル7（8つ目のラベル）のスニーカーであると予測できる

In [25]:
# 予測
model.predict(array)

1/1 [==============================] - 0s 67ms/step


array([[1.2442362e-09, 2.8949987e-10, 6.2720606e-11, 2.3186852e-09,
        9.9515389e-11, 7.5595141e-05, 5.6913231e-11, 9.9991369e-01,
        1.6403992e-07, 1.0557823e-05]], dtype=float32)